# Synchronized tdmclient interactive session

This notebook borrows largely from the repl help file of tdmclient.

First, make sure that tdmclient is installed for Jupyter:

In [ ]:
%pip install --upgrade tdmclient

Import the required class.

In [ ]:
import tdmclient.notebook

Connect to the TDM and start variable synchronization with the first robot:

In [ ]:
await tdmclient.notebook.start()

Read a Thymio variable:

In [ ]:
temp = temperature
print(temp)

Change a variable:

In [ ]:
leds_top = [0,32,32]

Write a program to be run on the Thymio:

In [ ]:
on = False
timer_period[0] = 500
@onevent
def timer0():
    global on, leds_top
    on = not on
    if on:
        leds_top = [32, 32, 0]
    else:
        leds_top = [0, 0, 0]

Run it. Actually it is not just the previous cell which is compiled to something the Thymio can execute, but all the functions decorated with `@onevent`, the functions they call, the Thymio variables which have been set, and other global variables they use.

In [ ]:
run()

Stop it:

In [ ]:
stop()

In [ ]:
You can also run on the Thymio a Python program contained in a single cell:

In [ ]:
%%run_python
v = [32, 0, 32, 0, 32, 0, 32, 0]
leds_circle = v

Or an Aseba program:

In [ ]:
%%run_aseba
var v[] = [32, 32, 32, 0, 0, 0, 32, 32]
leds.circle = v

The Python program is converted to Aseba (_transpiled_) before being compiled and executed on the Thymio. You can see the intermediate Aseba code:

In [ ]:
%%transpile_to_aseba
v = [32, 0, 32, 0, 32, 0, 32, 0]
leds_circle = v